In [31]:
import json
from public_data.models import Ocsge
from django.db import connection

from public_data.models import CommunesSybarval

qs = CommunesSybarval.objects.all()
comm = qs[3]
bbox = comm.mpoly.extent

# Initial parameters
# bbox = [-0.601487594874066, 44.6932226204681, -1.02920400564268, 44.4183678096998]



# generate initial SQL
query = (
    "SELECT id, couverture_label, usage_label, millesime, map_color, year, st_AsGeoJSON(mpoly, 4) AS geojson " 
    f"FROM {Ocsge._meta.db_table} "
    "WHERE mpoly && ST_MakeEnvelope(%s, %s, %s, %s, 4326) "
    "LIMIT 2"
)

query

'SELECT id, couverture_label, usage_label, millesime, map_color, year, st_AsGeoJSON(mpoly, 4) AS geojson FROM public_data_ocsge WHERE mpoly && ST_MakeEnvelope(%s, %s, %s, %s, 4326) LIMIT 2'

In [33]:
with connection.cursor() as cursor:
    cursor.execute(query, bbox)
    rows = cursor.fetchall()
rows

[(138353,
  'Zones non bâties (Routes; places; parking…)',
  'Routier',
  datetime.date(2018, 1, 1),
  None,
  2018,
  '{"type":"MultiPolygon","coordinates":[[[[-1.2022,44.6398],[-1.2022,44.6398],[-1.2022,44.6396],[-1.2023,44.6395],[-1.2023,44.6395],[-1.2023,44.6394],[-1.2023,44.6394],[-1.2023,44.6394],[-1.2023,44.6394],[-1.2023,44.6394],[-1.2023,44.6394],[-1.2023,44.6393],[-1.2023,44.6393],[-1.2023,44.6393],[-1.2024,44.6394],[-1.2024,44.6394],[-1.2024,44.6394],[-1.2024,44.6394],[-1.2025,44.6394],[-1.2025,44.6394],[-1.2025,44.6394],[-1.2025,44.6394],[-1.2025,44.6394],[-1.2026,44.6394],[-1.2027,44.6394],[-1.2027,44.6394],[-1.2028,44.6394],[-1.2028,44.6394],[-1.2028,44.6394],[-1.2028,44.6394],[-1.2028,44.6394],[-1.2028,44.6394],[-1.2029,44.6394],[-1.203,44.6394],[-1.203,44.6394],[-1.203,44.6395],[-1.203,44.6395],[-1.2031,44.6395],[-1.2032,44.6395],[-1.2032,44.6395],[-1.2032,44.6394],[-1.2032,44.6394],[-1.2033,44.6394],[-1.2033,44.6394],[-1.2034,44.6394],[-1.2034,44.6394],[-1.2035,44.6394

In [35]:
# generate features
features = []
for row in rows:
    feature = {
        'type': 'Feature',
        'properties': {
            'id': row[0],
            'couverture_label': row[1],
            'usage_label': row[2],
            'millesime': row[3].strftime("%Y-%m-%d"),
            'map_color': row[4],
            'year': row[5],
        },
        'geometry': json.loads(row[6])
    }
    features.append(feature)
features

[{'type': 'Feature',
  'properties': {'id': 138353,
   'couverture_label': 'Zones non bâties (Routes; places; parking…)',
   'usage_label': 'Routier',
   'millesime': '2018-01-01',
   'map_color': None,
   'year': 2018},
  'geometry': {'type': 'MultiPolygon',
   'coordinates': [[[[-1.2022, 44.6398],
      [-1.2022, 44.6398],
      [-1.2022, 44.6396],
      [-1.2023, 44.6395],
      [-1.2023, 44.6395],
      [-1.2023, 44.6394],
      [-1.2023, 44.6394],
      [-1.2023, 44.6394],
      [-1.2023, 44.6394],
      [-1.2023, 44.6394],
      [-1.2023, 44.6394],
      [-1.2023, 44.6393],
      [-1.2023, 44.6393],
      [-1.2023, 44.6393],
      [-1.2024, 44.6394],
      [-1.2024, 44.6394],
      [-1.2024, 44.6394],
      [-1.2024, 44.6394],
      [-1.2025, 44.6394],
      [-1.2025, 44.6394],
      [-1.2025, 44.6394],
      [-1.2025, 44.6394],
      [-1.2025, 44.6394],
      [-1.2026, 44.6394],
      [-1.2027, 44.6394],
      [-1.2027, 44.6394],
      [-1.2028, 44.6394],
      [-1.2028, 44.6394

In [11]:
# convert python dictionary into json
geojson = json.dumps({
    "type": "FeatureCollection",
    "crs": {"type": "name", "properties": {"name": f"EPSG:4326"}},
    'features': features
})

print(geojson)

{"type": "FeatureCollection", "crs": {"type": "name", "properties": {"name": "EPSG:4326"}}, "features": [{"type": "Feature", "properties": {"id": 76772, "couverture_label": "Zones non b\u00e2ties (Routes; places; parking\u2026)", "usage_label": "Routier", "millesime": "2018-01-01", "map_color": null, "year": 2018}, "geometry": {"type": "MultiPolygon", "coordinates": [[[[-0.9517, 44.6414], [-0.9517, 44.6414], [-0.9517, 44.6414], [-0.9516, 44.6414], [-0.9516, 44.6414], [-0.9516, 44.6415], [-0.9516, 44.6414], [-0.9516, 44.6414], [-0.9517, 44.6414], [-0.9517, 44.6414], [-0.9517, 44.6414], [-0.9517, 44.6414], [-0.9517, 44.6414]]]]}}, {"type": "Feature", "properties": {"id": 76816, "couverture_label": "Zones non b\u00e2ties (Routes; places; parking\u2026)", "usage_label": "Routier", "millesime": "2018-01-01", "map_color": null, "year": 2018}, "geometry": {"type": "MultiPolygon", "coordinates": [[[[-0.8832, 44.485], [-0.8833, 44.485], [-0.8833, 44.4848], [-0.8833, 44.4848], [-0.8833, 44.4848]

In [28]:


qs = CommunesSybarval.objects.all()
qs = qs.annotate(extent=Extent("mpoly"))
comm = qs.first()
print(comm.extent)
print(comm.mpoly.extent)

bbox = [-0.601487594874066, 44.6932226204681, -1.02920400564268, 44.4183678096998]

bbox = [comm.extent[2], comm.extent[3], comm.extent[0], comm.extent[1]]
print(bbox)

(-0.919067041179307, 44.4183677659233, -0.678639609235308, 44.5734589145821)
(-0.919067041179307, 44.4183677659233, -0.678639609235308, 44.5734589145821)
[-0.678639609235308, 44.5734589145821, -0.919067041179307, 44.4183677659233]


['CS1.1.1.1', '255', '55', '122']
['CS1.1.1.2', '255', '145', '145']
['CS1.1.2.1', '255', '255', '153']
['CS1.1.2.2', '166', '77', '0']
['CS1.2.1', '204', '204', '204']
['CS1.2.2', '0', '204', '242']
['CS1.2.3', '166', '230', '204']
['CS2.1.1.1', '128', '255', '0']
['CS2.1.1.2', '0', '166', '0']
['CS2.1.1.3', '128', '190', '0']
['CS2.1.2', '166', '255', '128']
['CS2.1.3', '230', '128', '0']
['CS2.2.1', '204', '242', '77']
['CS2.2.2', '204', '255', '204']
